# **Crawling Data**

In [ ]:
#@title Twitter Auth Token

#twitter_auth_token = '#ubah dengan auth token' # change this auth token'
#twitter_auth_token = '' # change this auth token
twitter_auth_token = '' # change this auth token

In [ ]:
# Import required Python package
!pip install pandas

# Install Node.js (because tweet-harvest built using Node.js)
!sudo apt-get update
!sudo apt-get install -y ca-certificates curl gnupg
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

!NODE_MAJOR=20 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

!sudo apt-get update
!sudo apt-get install nodejs -y

!node -v

In [ ]:
# Crawl Data

filename = 'demam_berdarah.csv'
#search_keyword = 'kesehatan near:Jakarta within:15km since:2025-01-01 lang:id'
search_keyword = 'demam berdarah since:2025-01-01 lang:id'
limit = 300

!npx -y tweet-harvest@2.6.1 -o "{filename}" -s "{search_keyword}" --tab "LATEST" -l {limit} --token {twitter_auth_token}

In [ ]:
import pandas as pd

# Specify the path to your CSV file
file_path = f"tweets-data/{filename}"


# Read the CSV file into a pandas DataFrame
#df = pd.read_csv(file_path, delimiter=",")
df = pd.read_csv("tweets-data/demam_berdarah.csv", delimiter=",")

# Display the DataFrame
display(df)

In [ ]:
# Cek jumlah data yang didapatkan

num_tweets = len(df)
print(f"Jumlah tweet dalam dataframe adalah {num_tweets}.")

## **PREPROCESSING**

https://journal.umkendari.ac.id/decode/article/view/131/61
1. Case Folding
2. Tokenizing
3. Stopword Removal
4. Normalisasi
5. Stemming

In [ ]:
#import nltk digunakan untuk mengimpor modul NLTK (Natural Language Toolkit) ke dalam program Python
#nltk.download('punkt') digunakan untuk mengunduh data yang diperlukan oleh tokenisasi Punkt dari NLTK.
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
import string
import re #regex library

# import word_tokenize & FreqDist from NLTK
# adalah dua fungsi yang diimpor dari modul NLTK (Natural Language Toolkit) dalam Python
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

# **CASE FOLDING**

In [ ]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
#untuk mengubah semua karakter dalam setiap elemen di dalam kolom (Series) menjadi huruf kecil (lowercase).
df['full_text_Case_Folding'] = df['full_text'].str.lower()


print('Case Folding Result : \n')
#print(TWEET_DATA['full_text'].head(5))
print(df['full_text_Case_Folding'].head(10))
print('\n\n\n')

In [ ]:
# Memecah setiap baris teks menjadi list kata-kata (token) berdasarkan spasi (split()).
# x.split() hanya memisahkan berdasarkan spasi (tidak hapus tanda baca, dll)
df["full_text_Case_Folding_split"] = df["full_text_Case_Folding"].apply(lambda x: len(x.split()))
df

# **CLEANSING**

**Tahapan Sebelum Preprocessing**

menghilangkan tweet special (seperti mention @username, hashtag #topik, URL, emoji, dan karakter khusus lainnya) biasanya merupakan bagian dari tahap preprocessing sebelum tokenizing, bukan bagian dari proses tokenizing itu sendiri. Namun, keduanya saling berkaitan.





In [ ]:
# NLTK word tokenize
# ungsi word_tokenize dalam NLTK (Natural Language Toolkit) adalah sebuah fungsi yang digunakan untuk membagi teks menjadi token-token kata individual, yang dikenal sebagai tokenisasi kata (word tokenization).
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")

df['full_text_cleansing'] = df['full_text_Case_Folding'].apply(remove_tweet_special)

#remove number
#def remove_number(text):: Ini adalah definisi fungsi yang disebut remove_number. Fungsi ini memiliki satu parameter yaitu text, yang merupakan teks yang akan diproses.
#return re.sub(r"\d+", "", text): Pada baris ini, fungsi re.sub() digunakan untuk mengganti setiap angka dalam teks dengan string kosong (menghapus angka).
# Baris terakhir menerapkan fungsi remove_number ke setiap elemen dalam kolom
# 'full_text' dalam objek DataFrame TWEET_DATA. Fungsi apply() digunakan untuk
# menerapkan fungsi ke setiap elemen dalam kolom atau baris DataFrame.

def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['full_text_cleansing'] = df['full_text_cleansing'].apply(remove_number)

def hapus_kata_underscore(teks):
    # Hapus kata yang mengandung underscore (baik awal, tengah, atau akhir)
    teks_baru = re.sub(r'\S*_\S*', '', teks)
    # Hapus spasi berlebih setelah penghapusan
    teks_baru = re.sub(r'\s+', ' ', teks_baru).strip()
    return teks_baru
df['full_text_cleansing'] = df['full_text_cleansing'].apply(hapus_kata_underscore)

#remove punctuation
#fungsi yang biasanya digunakan dalam pemrosesan teks untuk menghapus tanda baca dari suatu teks.
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['full_text_cleansing'] = df['full_text_cleansing'].apply(remove_punctuation)

# remove whitespace leading & trailing
# fungsi yang digunakan untuk menghapus spasi kosong (whitespace) pada bagian awal (leading) dan akhir (trailing) dari sebuah teks.
def remove_whitespace_LT(text):
    return text.strip()

df['full_text_cleansing'] = df['full_text_cleansing'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
#fungsi yang digunakan untuk mengganti beberapa spasi kosong berturut-turut menjadi satu spasi kosong tunggal dalam sebuah teks.
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['full_text_cleansing'] = df['full_text_cleansing'].apply(remove_whitespace_multiple)

# remove single char
# adalah sebuah fungsi yang digunakan untuk menghapus karakter tunggal (single character) dari sebuah teks.
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['full_text_cleansing'] = df['full_text_cleansing'].apply(remove_singl_char)

display(df)

In [ ]:
# Memecah setiap baris teks menjadi list kata-kata (token) berdasarkan spasi (split()).
# x.split() hanya memisahkan berdasarkan spasi (tidak hapus tanda baca, dll)
df["full_text_cleansing_split"] = df["full_text_cleansing"].apply(lambda x: len(x.split()))
df

# **TOKENIZING**

**Tokenizing**
adalah proses memecah teks atau string menjadi unit-unit yang lebih kecil yang disebut dengan token. Token bisa berupa kata-kata, frasa, simbol, karakter, atau unit lainnya tergantung pada tujuan dan konteks pengolahan teks.


In [ ]:
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['full_text_tokenizing'] = df['full_text_cleansing'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n')
print(df['full_text_tokenizing'].head())
print('\n\n\n')

In [ ]:
df.to_csv("Hasil-Preprocessing(Setelah Tokenizing).csv")

Fungsi **NLTK calc frequency distribution** digunakan untuk menghitung frekuensi distribusi kata dalam sebuah teks. Frekuensi distribusi adalah distribusi statistik yang menunjukkan jumlah kemunculan tiap elemen dalam kumpulan data.

Kelas Counter adalah sebuah kelas yang menyediakan fungsionalitas untuk menghitung dan mengelola elemen-elemen yang terdapat dalam suatu iterable (seperti list, string, atau tuple)

Setelah mengimpor Counter, kita dapat membuat objek Counter yang akan menghitung frekuensi kemunculan tiap elemen dalam iterable.

Fungsi freqDist_wrapper(text) adalah fungsi buatan pengguna (user-defined function).

Di dalam fungsi itu, kamu bisa menggunakan Counter (dari collections) untuk menghitung frekuensi kata atau token dalam teks.

Jadi, from collections import Counter bisa digunakan di dalam fungsi freqDist_wrapper() untuk menghitung distribusi frekuensi.



In [ ]:
from collections import Counter

# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

df['full_text_tokens_fdist'] = df['full_text_tokenizing'].apply(freqDist_wrapper)

print('Frequency Tokens : \n')
print(df['full_text_tokens_fdist'].head(10).apply(lambda x : x.most_common()))

# **STOPWORD REMOVAL**

In [ ]:
# digunakan untuk mengimpor modul stopwords dari NLTK (Natural Language Toolkit) dalam Python. Modul stopwords menyediakan daftar kata-kata yang umumnya dianggap sebagai kata-kata "stop words" dalam pemrosesan teks.

from nltk.corpus import stopwords

In [ ]:
#nltk.download('stopwords') digunakan untuk mengunduh data stop words (kata-kata yang umumnya dianggap sebagai kata-kata "stop words") dari NLTK. Stop words adalah kata-kata umum yang sering muncul dalam teks tetapi cenderung tidak memberikan informasi penting dalam pemrosesan teks, seperti kata-kata seperti "the", "is", "are", dan sebagainya.
nltk.download('stopwords')

In [ ]:
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
# kalo mau stopword berarti harus di tokenizing dulu, kalo tidak maka hasilnya beda
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
# adalah istilah yang digunakan untuk menambahkan kata-kata stop words tambahan ke dalam daftar stop words yang sudah ada.
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang',
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't',
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
# membaca file teks yang berisi daftar stop words menggunakan Pandas, Anda dapat menggunakan fungsi read_csv() dari Pandas dengan menggunakan pemisah (delimiter) yang sesuai.
txt_stopword = pd.read_csv("stopword.txt", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
# Untuk mengkonversi string stop words menjadi list dan menambahkan kata-kata stop words tambahan ke dalamnya
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
# mengonversi list list_stopwords menjadi set ist_stopwords.
list_stopwords = set(list_stopwords)


#remove stopword pada list token
#Dalam script ini, diasumsikan bahwa list_stopwords sudah didefinisikan sebelumnya. Anda perlu memastikan bahwa list_stopwords berisi kata-kata stop words yang sesuai dengan kebutuhan Anda sebelum menjalankan script ini.
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['full_text_tokens_Stopword'] = df['full_text_tokenizing'].apply(stopwords_removal)


print(df['full_text_tokens_Stopword'].head())

In [ ]:
df

# **NORMALISASI**

Normalisasi dalam text preprocessing adalah proses mengubah teks tidak baku atau tidak konsisten menjadi bentuk yang standar/baku, agar lebih mudah dianalisis oleh model NLP.



In [ ]:
# Dalam skrip ini, diasumsikan bahwa TWEET_DATA dan file Excel "normalisasi-V1.xlsx" sudah tersedia dan sesuai dengan struktur yang diharapkan.
normalizad_word = pd.read_excel("normalisasi-V1.xlsx")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['tweet_normalized'] = df['full_text_tokens_Stopword'].apply(normalized_term)

df['tweet_normalized'].head(10)

# **STEMMING**
Stemming adalah proses mengubah kata berimbuhan (infleksi atau turunan) menjadi bentuk dasar (stem)-nya. Tujuannya adalah menyederhanakan kata agar kata-kata dengan makna serupa dianggap sama dalam pemrosesan bahasa alami (NLP, seperti analisis sentimen, klasifikasi teks, dll).

In [ ]:
# pustaka sastrawi digunakan untuk melakukan stemming pada kata. StemmerFactory digunakan untuk membuat objek stemmer, dan kemudian metode stem() digunakan untuk melakukan stemming pada kata yang diberikan.
!pip install sastrawi

In [ ]:
# Fungsi Swifter dalam Pemrosesan Data
#Swifter adalah library Python yang dirancang untuk mempercepat pemrosesan data dengan memanfaatkan kekuatan multiprocessing.
!pip install swifter

In [ ]:
# import Sastrawi package
# mengimpor StemmerFactory dari Sastrawi untuk membuat objek stemmer. Kemudian, kita menggunakan swifter untuk menerapkan fungsi stemming pada kolom 'text' DataFrame menggunakan metode swifter.apply(). Hasil stemming disimpan dalam kolom baru 'stemmed_text'.
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
# menggunakan StemmerFactory dari pustaka Sastrawi untuk membuat objek stemmer. Kemudian, menggunakan metode stem() dari objek stemmer, kita melakukan stemming pada kata 'berjalan'. Hasil stemming akan dicetak, yaitu kata 'jalan'.
factory = StemmerFactory()
stemmer = factory.create_stemmer()


# stemmed
#stemmed_wrapper() adalah fungsi yang memanfaatkan objek stemmer untuk melakukan stemming pada setiap term. Kemudian, Anda melakukan iterasi pada term_dict dan menerapkan fungsi stemmed_wrapper()
# --pada setiap term untuk mendapatkan hasil stemming, yang kemudian disimpan kembali dalam term_dict. Terakhir, Anda mencetak term_dict yang berisi term-term yang telah distem.
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['tweet_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '

print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
# menggunakan swifter untuk mempercepat proses pemrosesan pada kolom 'tweet_normalized' dan menerapkan fungsi get_stemmed_term() pada setiap dokumen dalam kolom tersebut. Fungsi get_stemmed_term()
# --mengembalikan daftar term yang telah distem berdasarkan nilai yang ada dalam term_dict.
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['tweet_tokens_stemmed'] = df['tweet_normalized'].swifter.apply(get_stemmed_term)
print(df['tweet_tokens_stemmed'])

In [ ]:
df.to_csv("Hasil-Akhir-Preprocessing(Setelah Stemming).csv")

In [ ]:
df.to_csv("Hasil-Akhir-Normalisasi.csv")

In [ ]:
# menginstal pustaka openpyxl.
# Pustaka ini merupakan salah satu pustaka populer untuk membaca dan menulis file berformat Excel (XLSX) menggunakan Python.
!pip install openpyxl

In [ ]:
# menginstal pustaka xlrd.
# Pustaka ini digunakan untuk membaca file berformat Excel (XLS) menggunakan Python.
!pip install xlrd

In [ ]:
!pip install nltk

In [ ]:
from nltk.tokenize import sent_tokenize


In [ ]:
df

# **Menggabungkan Kalimat Hasil Tokenizing**

In [ ]:
df['kalimat_tanpa_kurung'] = df['tweet_tokens_stemmed'].apply(lambda x: ' '.join(x))

In [ ]:
df

In [ ]:
df.to

**Menghitung Jumlah Kata**

In [ ]:
#Cara Ke-1
df["kalimat_tanpa_kurung_split"] = df["kalimat_tanpa_kurung"].apply(lambda x: len(x.split()))
df

#Cara Ke-2
# Hitung jumlah kata (token) dalam setiap baris (kalimat) pada kolom teks.
# word_tokenize(x) (dari nltk) lebih canggih: mengenali tanda baca dan format bahasa
# Fungsi ini tidak bisa dilakukan pada Hasil Tokenisasi (atau ada koma atau petik)
df["JumlahKata"] = df["kalimat_tanpa_kurung"].apply(lambda x: len(word_tokenize(x)))
df

# **Leksikon**

In [ ]:
pip install vaderSentiment

In [ ]:
pip install numpy

In [ ]:
pip install scikit-learn

In [ ]:
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
# Create an instance of SentimentIntensityAnalyzer
model = SentimentIntensityAnalyzer()

In [ ]:
import csv
import os
#find file path for lexicon bahasa indonesia
#filepath = '/sembako4.csv'

#create dictionary for positive lexicon
lexicon_positive = {}

#read the positive tsv file
with open(os.path.join('positive.tsv')) as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    next(reader)
    for word, weight in reader:
        lexicon_positive[word] = int(weight)

#create dictionary for negative lexicon
lexicon_negative = {}

#read the negative tsv file
with open(os.path.join('negative.tsv')) as tsv_file:
    reader = csv.reader(tsv_file, delimiter='\t')
    next(reader)
    for word, weight in reader:
        lexicon_negative[word] = int(weight)

#check the dictionaries
print(lexicon_positive)
print(lexicon_negative)

In [ ]:
df

Untuk menambahkan tanda petik (") atau tanda kutip (') ke dalam kolom DataFrame di Python (misalnya di sekitar setiap kata atau kalimat), kamu bisa menggunakan fungsi apply() atau str.replace() atau str.join() tergantung bentuk datanya.

Berikut beberapa contoh umum:

In [ ]:
#select columns called 'Text Filtering'
df['tweet_tokens_stemmed_2'] = df['tweet_tokens_stemmed'].apply(lambda x: f'{x}')

#view new DataFrame
df

error seperti ini karena data frame tidak ada tanda kutip
cek https://github.com/agushendra7/twitter-sentiment-analysis-using-vader-and-random-forest/blob/main/labeling/vader%20sentiment.ipynb

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

df['Score'] = df['tweet_tokens_stemmed'].apply(lambda x: vs.polarity_scores(x))
df['Compound Score'] = df['tweet_tokens_stemmed'].apply(lambda x: vs.polarity_scores(x)['compound'])

df.head()

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs = SentimentIntensityAnalyzer()

df['Score'] = df['tweet_tokens_stemmed_2'].apply(lambda x: vs.polarity_scores(x))
df['Compound Score'] = df['tweet_tokens_stemmed_2'].apply(lambda x: vs.polarity_scores(x)['compound'])

df.head()

In [ ]:
def vader_analysis(compound):
    if compound >= 0.05:
        return 'Positive'
    elif compound <=  -0.05:
        return 'Negative'
    else:
        return 'Neutral'

df['Vader Sentiment'] = df['Compound Score'].apply(vader_analysis)
df.head(10)

In [ ]:
vader_counts = df['Vader Sentiment'].value_counts()
vader_counts

In [ ]:
df.to_csv("bencana-alam-sentiment.csv")

# **Disini Untuk Mengetahui Sentimen Hasil**

In [ ]:
df

In [ ]:
df['kalimat_tanpa_kurung'] = df['tweet_tokens_stemmed'].apply(lambda x: ' '.join(x))

In [ ]:
df

In [ ]:
# Modul pandas digunakan untuk manipulasi dan analisis data tabular, sedangkan modul Counter digunakan untuk menghitung kemunculan elemen-elemen dalam suatu koleksi.
import pandas as pd
from collections import Counter

# **Menghitung Banyak Kata**

In [ ]:
# Memisahkan kata-kata dalam teks
kata_kunci = df['kalimat_tanpa_kurung'].str.split()
kata_kunci

In [ ]:
df.info()

# **Mengubah Nama Kolom**

In [ ]:
df.rename(columns={'Vader Sentiment': 'label'}, inplace=True)
df.head()

In [ ]:
import numpy as np
import pandas as pd
import nltk
from io import StringIO
import pytz
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import ast
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
from nltk.tokenize import SpaceTokenizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
import os
import socket
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
#st.set_option('deprecation.showPyplotGlobalUse', False) #ini dikasih komentar biar ga error
import math
import pprint
from sklearn.svm import LinearSVC #classifier SVM
from sklearn.svm import SVC
# Menginisialisasi Sastrawi Stemmer
from wordcloud import WordCloud
factory = StemmerFactory()
stemmer = factory.create_stemmer()


In [ ]:
"""TF-IDF"""
#Perlu dicari cara untuk membuat Label dengan TF-IDF

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(df['kalimat_tanpa_kurung'], df['label'], test_size=0.2, random_state=7)

df_train = pd.DataFrame()
df_train['kalimat_tanpa_kurung'] = x_train
df_train['label'] = y_train

df_test= pd.DataFrame()
df_test['kalimat_tanpa_kurung'] = x_test
df_test['label'] = y_test

df_train

df_test

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf vectorizer
tfidf=TfidfVectorizer()
tfidf.fit(df['kalimat_tanpa_kurung'].values.astype('U'))
#hitung tf-idf setiap kata pada data training
x_train_tfidf = tfidf.transform(df_train['kalimat_tanpa_kurung'].values.astype('U'))
#hitung tf-idf setiap kata pada data testing
x_test_tfidf = tfidf.transform(df_test['kalimat_tanpa_kurung'].values.astype('U'))

df_train.to_csv('tweet-train.csv', index=False)
df_train.to_csv('tweet-test.csv', index=False)

tfidf

print(x_train_tfidf)

print(x_test_tfidf)

print(x_train_tfidf.shape)
print(x_test_tfidf.shape)

print(tfidf.vocabulary_)

# **SVM**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# Split data
x = df['kalimat_tanpa_kurung']
y = df['label']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

In [ ]:
SVM = SVC(kernel='linear')  # Gunakan kernel linear untuk teks
SVM.fit(x_train_tfidf, y_train)

In [ ]:
# Prediksi
predictions_SVM = SVM.predict(x_test_tfidf)

# Akurasi
accuracy = accuracy_score(y_test, predictions_SVM)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Confusion Matrix
matrix = confusion_matrix(y_test, predictions_SVM)
print("Confusion Matrix:\n", matrix)

# Heatmap
sns.heatmap(matrix, annot=True, fmt='g', cmap='Blues',
            xticklabels=['Positif', 'Netral', 'Negatif'],
            yticklabels=['Positif', 'Netral', 'Negatif'])
plt.xlabel('Prediksi')
plt.ylabel('Asli')
plt.title('Confusion Matrix - SVM')
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_test, predictions_SVM))

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions_SVM)
print("Accuracy:", accuracy)


**average	Penjelasan**

'micro'	Menghitung total TP, FP, FN dari seluruh kelas (global)
- Ingin skor global akurat, cocok untuk data seimbang

'macro'	Rata-rata metrik dari tiap kelas (semua kelas dianggap sama penting)
- Ingin adil ke semua kelas, cocok untuk data tidak seimbang

'weighted'	Rata-rata metrik tiap kelas, diberi bobot sesuai jumlah sampel tiap kelas
- Kompromi antara micro dan macro

None	Mengembalikan skor per kelas (array)
- Butuh skor per kelas secara terpisah

In [ ]:
# Calculate precision
precision = precision_score(y_test, predictions_SVM, average='macro')
print("Precision:", precision)

# Calculate recall (sensitivity)
recall = recall_score(y_test, predictions_SVM, average='macro')
print("Recall (Sensitivity):", recall)

# Calculate F1-score
f1 = f1_score(y_test, predictions_SVM, average='macro')
print("F1-Score:", f1)

In [ ]:
#Simpan Hasil Prediksi (Opsional)
test_prediction = pd.DataFrame()
test_prediction['text'] = x_test
test_prediction['label'] = predictions_SVM
test_prediction.to_csv('svm_sentiment_predictions.csv', index=False)

# **RANDOM FOREST**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# Pisahkan fitur dan label
x = df['kalimat_tanpa_kurung']
y = df['label']

# Split data menjadi train dan test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(x_train_tfidf, y_train)

In [ ]:
# Prediksi
predictions_RF = rf_model.predict(x_test_tfidf)

# Akurasi
accuracy = accuracy_score(y_test, predictions_RF)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Confusion Matrix
matrix = confusion_matrix(y_test, predictions_RF)
print("Confusion Matrix:\n", matrix)

# Heatmap Confusion Matrix
sns.heatmap(matrix, annot=True, fmt='g', cmap='YlGnBu',
            xticklabels=['Positif', 'Netral', 'Negatif'],
            yticklabels=['Positif', 'Netral', 'Negatif'])
plt.xlabel('Prediksi')
plt.ylabel('Label Sebenarnya')
plt.title('Confusion Matrix - Random Forest')
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_test, predictions_RF))

In [ ]:
#Simpan Hasil Prediksi (Opsional)
# Simpan prediksi ke file CSV
test_prediction = pd.DataFrame()
test_prediction['text'] = x_test
test_prediction['label'] = predictions_RF
test_prediction.to_csv('randomforest_sentiment_predictions.csv', index=False)

# **Naive Bayes**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
# Pisahkan fitur dan label
x = df['kalimat_tanpa_kurung']
y = df['label']

# Split train dan test set
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
tfidf = TfidfVectorizer(max_features=5000)
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)

In [ ]:
nb_model = MultinomialNB()
nb_model.fit(x_train_tfidf, y_train)

In [ ]:
# Prediksi
predictions_NB = nb_model.predict(x_test_tfidf)

# Akurasi
accuracy = accuracy_score(y_test, predictions_NB)
print("Accuracy: {:.2f}%".format(accuracy * 100))

# Confusion Matrix
matrix = confusion_matrix(y_test, predictions_NB)
print("Confusion Matrix:\n", matrix)

# Visualisasi confusion matrix
sns.heatmap(matrix, annot=True, fmt='g', cmap='coolwarm',
            xticklabels=['Positif', 'Netral', 'Negatif'],
            yticklabels=['Positif', 'Netral', 'Negatif'])
plt.xlabel('Prediksi')
plt.ylabel('Label Sebenarnya')
plt.title('Confusion Matrix - Naive Bayes')
plt.show()

# Classification Report
print("Classification Report:")
print(classification_report(y_test, predictions_NB))

**Lain lagi**

In [ ]:
# metode naive bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
bayes = MultinomialNB()
##tahapan untuk menjalankan training
bayes.fit(x_train_tfidf,y_train)

#menjalankan data testing
nb_result = bayes.predict(x_test_tfidf)
from sklearn import metrics
model = metrics.accuracy_score(y_test, nb_result)
print("The Accuracy is",str('{:04.2f}'.format(model*100))+'%')

from sklearn.metrics import confusion_matrix
matrix2= confusion_matrix(y_test, nb_result)
print(matrix2)

import seaborn as sns
sns.heatmap(matrix2, square= True, annot= True, cbar= False, cmap='RdBu', xticklabels=['Positif','Netral','Negatif'], yticklabels= ['Positif','Netral','Negatif'], fmt= 'g')
plt.xlabel('prediksi label')
plt.ylabel('true label')

from sklearn.metrics import classification_report
print("The classification report is:")
print(classification_report(y_test, nb_result))

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, nb_result)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_test, nb_result, average='macro')
print("Precision:", precision)

# Calculate recall (sensitivity)
recall = recall_score(y_test, nb_result, average='macro')
print("Recall (Sensitivity):", recall)

# Calculate F1-score
f1 = f1_score(y_test, nb_result, average='macro')
print("F1-Score:", f1)

In [ ]:
# hasil

Berita = input('Masukkan Tweet : ')
result = bayes.predict(tfidf.transform([Berita]))
if(result == [1]):
  result = 'Berita yang dimasukkan merupakan berita Gempa Bumi'
elif(result==[4]):
  result = 'Berita yang dimasukkan merupakan berita Banjir'
result